In [1]:
import pandas as pd
from scripts.utils import *
import concurrent.futures

In [2]:
df = pd.read_csv("data/raw_data_from_sana.csv")
df.head()

,chr,start,ref,alt,driver
0,16,339439,C,T,1
1,16,339608,C,T,1
2,16,347721,C,T,1
3,16,360070,C,A,1
4,16,396146,A,T,1


https://www.ncbi.nlm.nih.gov/clinvar/RCV000140032/ this page shows that this variant chromosome "Un_KI270742v1" is at Chr1: 142728921 - 142907112 (on Assembly GRCh37)

so I added the line 

```df.loc[df['chr'] == 'Un_KI270742v1', 'chr'] = '1'```

# processing df

In [3]:
df.loc[df['chr'] == 'Un_KI270742v1', 'chr'] = '1'

In [4]:
# renaming cols
df.rename(columns={"start": "pos"}, inplace=True)

In [5]:
# handle minus signs
df["ref"] = df["ref"].replace("-", "")
df["alt"] = df["alt"].replace("-", "")

In [6]:
# Add an 'id' column by combining 'chr', 'pos', 'ref', and 'alt' columns
df['id'] = df['chr'] + '_' + df['pos'].astype(str) + '_' + df['ref'] + '_' + df['alt']

In [7]:
# preparing mutation len values
df.loc[:, "ref_len"] = df["ref"].str.len()
df.loc[:, "alt_len"] = df["alt"].str.len()

In [8]:
# if affected sequences are longer than 1, fetch nucleotides at the specified interval
df.loc[df['ref_len'] > 1, 'fetched_nucleotides'] = df.apply(lambda x: get_nucleotides_in_interval(x['chr'], x['pos'], x["pos"]+x["ref_len"]-1), axis=1)
# if affected sequences are equal to 1, fetch nucleotides at the specified pos
df.loc[df['ref_len'] == 1, 'fetched_nucleotides'] = df.apply(lambda x: get_nucleotide_at_position(x['chr'], x['pos']), axis=1)
# same but for 0 length
df.loc[df['ref_len'] == 0, 'fetched_nucleotides'] = ""

In [9]:
# Add an 'is_nucleotides_same' column to check if fetched nucleotides are the same as 'ref'
df.loc[:, 'is_nucleotides_same'] = df["fetched_nucleotides"] == df["ref"]
df.is_nucleotides_same.value_counts()


is_nucleotides_same
True    1154
Name: count, dtype: int64

In [10]:
# preparing reference indices
df.loc[:, "ref_start"] = df["pos"]
df.loc[:, "ref_end"] = df["pos"] + df["ref_len"]


In [11]:
# preparing extended sequences
df.loc[:, 'upstream_sequence'] = df.apply(lambda x: get_nucleotides_in_interval(x['chr'], x['ref_start']-30, x["ref_start"]-1), axis=1)
df.loc[:, 'downstream_sequence'] = df.apply(lambda x: get_nucleotides_in_interval(x['chr'], x['ref_end'], x["ref_end"]+29), axis=1)

In [12]:
# preparing wt sequence
df.loc[:,'sequence'] = df["upstream_sequence"] + df["ref"] + df["downstream_sequence"]

# preparing mutated sequence
df.loc[:,'mutated_sequence'] = df["upstream_sequence"] + df["alt"] + df["downstream_sequence"]

In [13]:
df = generate_is_mirna_column(df, grch=37)
df = generate_transcript_id_and_gene_name_columns(df, grch=37)


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


In [14]:
df[df.is_mirna == 1].head()

,chr,pos,ref,alt,driver,id,ref_len,alt_len,fetched_nucleotides,is_nucleotides_same,ref_start,ref_end,upstream_sequence,downstream_sequence,sequence,mutated_sequence,is_mirna,transcript_id,gene_name
382,17,56408612,A,T,1,17_56408612_A_T,1,1,A,True,56408612,56408613,TCTCCGAAGCCCACAGTACACTCATCCATA,AGTAGGAAACACTACACCCTCCAGTGCTGT,TCTCCGAAGCCCACAGTACACTCATCCATAAAGTAGGAAACACTAC...,TCTCCGAAGCCCACAGTACACTCATCCATATAGTAGGAAACACTAC...,1,"[ENST00000384835, ENST00000578334, ENST0000057...","[BZRAP1-AS1, MIR142]"
383,17,56408616,A,C,1,17_56408616_A_C,1,1,A,True,56408616,56408617,CGAAGCCCACAGTACACTCATCCATAAAGT,GGAAACACTACACCCTCCAGTGCTGTTAGT,CGAAGCCCACAGTACACTCATCCATAAAGTAGGAAACACTACACCC...,CGAAGCCCACAGTACACTCATCCATAAAGTCGGAAACACTACACCC...,1,"[ENST00000384835, ENST00000578334, ENST0000057...","[BZRAP1-AS1, MIR142]"
384,17,56408620,A,G,1,17_56408620_A_G,1,1,A,True,56408620,56408621,GCCCACAGTACACTCATCCATAAAGTAGGA,ACACTACACCCTCCAGTGCTGTTAGTAGTG,GCCCACAGTACACTCATCCATAAAGTAGGAAACACTACACCCTCCA...,GCCCACAGTACACTCATCCATAAAGTAGGAGACACTACACCCTCCA...,1,"[ENST00000384835, ENST00000578334, ENST0000057...","[BZRAP1-AS1, MIR142]"
385,17,56408620,AA,TG,1,17_56408620_AA_TG,2,2,AA,True,56408620,56408622,GCCCACAGTACACTCATCCATAAAGTAGGA,CACTACACCCTCCAGTGCTGTTAGTAGTGC,GCCCACAGTACACTCATCCATAAAGTAGGAAACACTACACCCTCCA...,GCCCACAGTACACTCATCCATAAAGTAGGATGCACTACACCCTCCA...,1,"[ENST00000384835, ENST00000578334, ENST0000057...","[BZRAP1-AS1, MIR142]"
386,17,56408621,A,G,1,17_56408621_A_G,1,1,A,True,56408621,56408622,CCCACAGTACACTCATCCATAAAGTAGGAA,CACTACACCCTCCAGTGCTGTTAGTAGTGC,CCCACAGTACACTCATCCATAAAGTAGGAAACACTACACCCTCCAG...,CCCACAGTACACTCATCCATAAAGTAGGAAGCACTACACCCTCCAG...,1,"[ENST00000384835, ENST00000578334, ENST0000057...","[BZRAP1-AS1, MIR142]"


In [15]:
def get_biotype(coord):
    if transcripts := grch37.transcripts_at_locus(*coord):
        return tuple(coord), transcripts[0].biotype
    else:
        return tuple(coord), "not_found"
    
grch37 = import_pyensembl(grch=37)
coords = df[['chr', 'pos']].values.tolist()

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(get_biotype, coords)

biotypes = dict(results)
df["biotype"] = [biotypes.get((row["chr"], row["pos"]), "") for _, row in df.iterrows()]


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


In [16]:
df.biotype.value_counts()

biotype
protein_coding             740
not_found                  319
misc_RNA                    20
miRNA                       17
processed_transcript        16
nonsense_mediated_decay     15
lincRNA                      8
antisense                    8
snoRNA                       4
retained_intron              4
IG_C_gene                    1
unprocessed_pseudogene       1
sense_overlapping            1
Name: count, dtype: int64

In [17]:
df[df.biotype == "miRNA"].is_mirna.value_counts()

is_mirna
1    11
0     6
Name: count, dtype: int64

In [18]:
df.head()

,chr,pos,ref,alt,driver,id,ref_len,alt_len,fetched_nucleotides,is_nucleotides_same,ref_start,ref_end,upstream_sequence,downstream_sequence,sequence,mutated_sequence,is_mirna,transcript_id,gene_name,biotype
0,16,339439,C,T,1,16_339439_C_T,1,1,C,True,339439,339440,GGGGAGGGGGCACCCCAGCCCTCACACTCA,CTGTAGCTGCCCTTTTTGGTCAGCAGCTCC,GGGGAGGGGGCACCCCAGCCCTCACACTCACCTGTAGCTGCCCTTT...,GGGGAGGGGGCACCCCAGCCCTCACACTCATCTGTAGCTGCCCTTT...,0,"[ENST00000262320, ENST00000354866, ENST0000045...",[AXIN1],protein_coding
1,16,339608,C,T,1,16_339608_C_T,1,1,C,True,339608,339609,CCGCCGCCCACCTTCCTCTGCGATCTTGTC,TGGGGAAAGAGATGCAGCGGTGGTACCTGG,CCGCCGCCCACCTTCCTCTGCGATCTTGTCCTGGGGAAAGAGATGC...,CCGCCGCCCACCTTCCTCTGCGATCTTGTCTTGGGGAAAGAGATGC...,0,"[ENST00000262320, ENST00000354866, ENST0000045...",[AXIN1],protein_coding
2,16,347721,C,T,1,16_347721_C_T,1,1,C,True,347721,347722,GCCCCCTCCTCACTGACAGGCGCACGCTCA,CTGTGGGCGAGGCCATCACTGGCGTTGGGG,GCCCCCTCCTCACTGACAGGCGCACGCTCACCTGTGGGCGAGGCCA...,GCCCCCTCCTCACTGACAGGCGCACGCTCATCTGTGGGCGAGGCCA...,0,"[ENST00000262320, ENST00000354866, ENST0000046...",[AXIN1],protein_coding
3,16,360070,C,A,1,16_360070_C_A,1,1,C,True,360070,360071,TTACGGATCCTGTATGGGGGGATCCCATCC,TGTCCAGGAGAAAGAGGCAGCCGTTAACTC,TTACGGATCCTGTATGGGGGGATCCCATCCCTGTCCAGGAGAAAGA...,TTACGGATCCTGTATGGGGGGATCCCATCCATGTCCAGGAGAAAGA...,0,"[ENST00000262320, ENST00000354866, ENST0000046...",[AXIN1],protein_coding
4,16,396146,A,T,1,16_396146_A_T,1,1,A,True,396146,396147,TACAGAAAGTGGACGCCTGGCGTCGGACTC,CCTGAACTCTCTGCCTTCGCTGTACCGTCT,TACAGAAAGTGGACGCCTGGCGTCGGACTCACCTGAACTCTCTGCC...,TACAGAAAGTGGACGCCTGGCGTCGGACTCTCCTGAACTCTCTGCC...,0,"[ENST00000262320, ENST00000354866, ENST0000046...",[AXIN1],protein_coding


In [19]:
df[(df.biotype == "miRNA") & (df.is_mirna == 0)].transcript_id.values

array([list(['ENST00000384835', 'ENST00000578334', 'ENST00000579003', 'ENST00000579527', 'ENST00000580515', 'ENST00000580633']),
       list(['ENST00000384835', 'ENST00000578334', 'ENST00000579003', 'ENST00000579527', 'ENST00000580515', 'ENST00000580633']),
       list(['ENST00000384835', 'ENST00000578334', 'ENST00000579003', 'ENST00000579527', 'ENST00000580515', 'ENST00000580633']),
       list(['ENST00000384835', 'ENST00000578334', 'ENST00000579003', 'ENST00000579527', 'ENST00000580515', 'ENST00000580633']),
       list(['ENST00000384835', 'ENST00000578334', 'ENST00000579003', 'ENST00000579527', 'ENST00000580515', 'ENST00000580633']),
       list(['ENST00000384835', 'ENST00000578334', 'ENST00000579003', 'ENST00000579527', 'ENST00000580515', 'ENST00000580633'])],
      dtype=object)

In [20]:
case_1 = df[df.is_mirna == 0]
case_2 = df[df.is_mirna == 1]

case_1.to_csv("data/case_1_processed.csv", index=False)
case_2.to_csv("data/case_2_processed.csv", index=False)